In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
import random
from xgboost import XGBClassifier
from sklearn.metrics import precision_score
import yfinance as yf
from sklearn.model_selection import train_test_split





In [2]:
sp500 = yf.download('^GSPC',start = '2000-01-01',interval='1d')
sp500["Tomorrow"] = sp500["Close"].shift(-1)
sp500["Target"] = (sp500["Tomorrow"] > sp500["Close"]).astype(int)
sp500 = sp500.drop(columns=['Adj Close'])



[*********************100%%**********************]  1 of 1 completed


In [3]:
train = sp500.iloc[:-100]
test = sp500.iloc[-100:]
train, validation = train_test_split(train, test_size=0.2, shuffle=False)


etas = [0.01,0.05,0.10,0.20,0.30,0.40,0.5,0.6,0.7]

params = {
    'max_depth': [5, 10, 15, 20,25],
    'subsample': [0.5,0.6,0.7,0.8,0.9,1.0],
    'colsample_bytree': [0.5,0.6,0.7,0.8,0.9,1.0],
    'gamma' : [0.00,0.05,0.10,0.15,0.20],
    'scale_pos_weight' : [30,40,50,300,400,500,600,700], 
}

iSol = {}
for key,values in params.items():
    iSol[key] = random.choice(values)

cSol = iSol
tStart = 100
tEnd = 1e-6
nLoops = 30
#overfit_threshold = 0.05 


frac = (tEnd/tStart)**(1.0/(nLoops-1.0))





def predict(train, test, predictors, model):
    model.fit(train[predictors], train["Target"])
    preds = model.predict(test[predictors])
    preds = pd.Series(preds, index=test.index, name="Predictions")
    combined = pd.concat([test["Target"], preds], axis=1)
    return combined


def backtest(data, model, predictors, start=2500, step=250):
    all_predictions = []

    for i in range(start, data.shape[0], step):
        train = data.iloc[0:i].copy()
        test = data.iloc[i:(i+step)].copy()
        predictions = predict(train, test, predictors, model)
        all_predictions.append(predictions)
    
    return pd.concat(all_predictions)





def objective_function(cSol):
    
    model = XGBClassifier(**cSol)
    
    
    predictors = ["Close", "Volume", "Open", "High", "Low"]
    model.fit(train[predictors], train["Target"])
    
    
    predictions = backtest(sp500, model, predictors)
    score = precision_score(predictions["Target"], predictions["Predictions"])

    train_preds = model.predict(train[predictors])
    train_accuracy = precision_score(train["Target"], train_preds)

    
    valid_preds = model.predict(validation[predictors])
    valid_accuracy = precision_score(validation["Target"], valid_preds)


    overfit_measure = train_accuracy - valid_accuracy
    
    return score,overfit_measure


def simulated_annealing(T, frac, n):
    
    current_solution = {}

    

    for key,values in params.items():
        current_solution[key] = random.choice(values)
        current_solution.update({'eta':0.001})

    
    current_obj_val,overfit_measure = objective_function(current_solution)
    
    
    for i in range(n):
        
        new_solution = {}
        for  eta in etas:
            for key,values in params.items():
                new_solution[key] = random.choice(values)
                new_solution.update({'eta':eta})
            
            
            new_obj_val,overfit_measure= objective_function(new_solution)

            if (overfit_measure < 0.015) and(overfit_measure > -0.015):

                if new_obj_val>current_obj_val:
                    current_obj_val = new_obj_val
                    current_solution = new_solution
                    
                elif new_obj_val <= current_obj_val:
                    
                    p = np.exp(-(current_obj_val - new_obj_val) / T)
                    
                    if np.random.uniform(0, 1) < p:
                        current_solution = new_solution
                        current_obj_val = new_obj_val
                        
                
                
                T *= frac
                
                
                if T < tEnd:
                    break
                print('Current Overfit Measure : ',overfit_measure)
                print("Current Temp", T)
                print("Current Hyperparameters: ",current_solution)
                print("Current Objective Value: ",current_obj_val)
            
    

    return current_solution, current_obj_val,T,overfit_measure


best_solution, best_obj_val,T,overfit_measure = simulated_annealing(tStart, frac, nLoops)


print("Best Hyperparameters : ", best_solution)
print("Best Objective Function Value : ",best_obj_val)
print('Final Overfit Measure : ',overfit_measure)
print("Final Temp", T)




Current Overfit Measure :  -0.004822285902925216
Current Temp 52.983169062837085
Current Hyperparameters:  {'max_depth': 25, 'eta': 0.01, 'subsample': 0.5, 'colsample_bytree': 1.0, 'gamma': 0.05, 'scale_pos_weight': 40}
Current Objective Value:  0.5430669302456933
Current Overfit Measure :  -0.0102772496435094
Current Temp 28.07216203941177
Current Hyperparameters:  {'max_depth': 20, 'eta': 0.05, 'subsample': 1.0, 'colsample_bytree': 0.9, 'gamma': 0.05, 'scale_pos_weight': 30}
Current Objective Value:  0.5397030342156229
Current Overfit Measure :  -0.009178879622213865
Current Temp 14.873521072935112
Current Hyperparameters:  {'max_depth': 5, 'eta': 0.1, 'subsample': 0.7, 'colsample_bytree': 1.0, 'gamma': 0.2, 'scale_pos_weight': 40}
Current Objective Value:  0.5425052568338841
Current Overfit Measure :  -0.010860641859804399
Current Temp 7.880462815669912
Current Hyperparameters:  {'max_depth': 5, 'eta': 0.3, 'subsample': 0.8, 'colsample_bytree': 0.8, 'gamma': 0.0, 'scale_pos_weight':